# Hybrid RAG Demo: FAISS + BM25


This notebook demonstrates a **Hybrid RAG** pipeline that uses:
- **BM25** for lexical matching (exact word matches)
- **FAISS** for semantic similarity using Sentence Transformers
- **Rank Fusion** to combine both methods for better retrieval quality


In [1]:

!pip install rank_bm25 faiss-cpu sentence-transformers langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:

from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import random


In [3]:

corpus = [
    "I want to lose weight and eat better.",
    "Sometimes I eat fast food because it's convenient.",
    "I’ve tried dieting before but gave up quickly.",
    "I know I should exercise more but I feel too tired.",
    "My family habits make it hard to change.",
    "I want to be healthier for my kids.",
    "Obesity is affecting my self-confidence.",
    "Walking is one thing I enjoy doing.",
    "I feel judged when I talk about my weight.",
    "I want to feel good about my body."
]


In [4]:

tokenized_corpus = [doc.lower().split() for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)


In [5]:

model = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = model.encode(corpus, convert_to_numpy=True)

index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:

def hybrid_retrieve(query, k=3, alpha=0.5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    D, I = index.search(query_embedding, k)

    bm25_scores = bm25.get_scores(query.lower().split())
    bm25_ranks = np.argsort(bm25_scores)[::-1]

    bm25_top = bm25_ranks[:k]
    faiss_top = I[0]

    # Normalize and fuse ranks
    score_dict = {}
    for idx in bm25_top:
        score_dict[idx] = score_dict.get(idx, 0) + alpha * bm25_scores[idx]
    for i, idx in enumerate(faiss_top):
        score_dict[idx] = score_dict.get(idx, 0) + (1 - alpha) * (1 - D[0][i])

    ranked = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
    return [(corpus[idx], score) for idx, score in ranked[:k]]


In [7]:

query = "I want to stop eating junk food"
top_docs = hybrid_retrieve(query)
for i, (doc, score) in enumerate(top_docs, 1):
    print(f"{i}. {doc} (score={score:.4f})")


1. Sometimes I eat fast food because it's convenient. (score=1.1300)
2. I want to lose weight and eat better. (score=0.8675)
3. I want to feel good about my body. (score=0.7720)



This notebook shows how to:
- Combine **BM25** lexical scores with **FAISS** semantic similarity
- Run a rank fusion for hybrid retrieval
- Adapt to your RAG pipeline by replacing corpus with real data
